In [8]:
import pandas as pd
from datetime import datetime
import time
import requests

In [9]:
tokenApi = ['enb4iHROSDegvkiZv17FxzISZ%2BZB%2FQxkSGweAsL8sB0%3D']
url = "https://elysia.zeev.it"
tipoUrl = "/api/2/assignments"

# Periodo de selecao
today = datetime.today().strftime("%Y-%m-%d")
periodo1 = "2023-03-17"
periodo2 = "2023-03-10"

In [10]:
# Se a API alterar seu campos novamente, sera necessario alterar apenas alguns filtros desta funcao
def request_Orquestra(method='GET',urlReq="https://elysia.zeev.it",typeReq="/api/2/assignments",tokens=[],filters={}, forceRequisition:bool=False) -> list:
    # Variaveis de controle
    row = 1
    i = 1
    retorno = []
    # filtros base para o payload
    print("Requisicao para {} <<".format(urlReq+typeReq))
    if not typeReq.lower().__contains__('report'):
        filters['recordsPerPage'] = 100
        seconds = 1
    else:
        if not forceRequisition: 
            print("---- >> Requisicao lenta... << ---- ")
            filters['recordsPerPage'] = 30
            seconds = 3
        else:
            print("---- >> Requisicao forçada... << ---- ")
            filters['recordsPerPage'] = 100
            seconds = 1    
    
    filters['mobileEnabledOnly'] = False
    # Faz a busca para cada token
    for tk in tokens:
        # Reseta variaveis de controle
        row = 1
        i = 1
        # Iteracao com token
        authorization= {'Authorization': "Bearer " + tk[0]}
        while row != 0:
            # Aguardo x segundos devido ao limite de requisicoes - lembrando que a limite de requisicoes por segundos, minutos, horas, dias e meses
            time.sleep(seconds) 
            
            # Incremento de pagina
            filters['pageNumber'] = i
            
            # Requisicao
            print(filters)
            getReq = requests_API_Orquestra(metodo=method, 
                                            urlAcesso=urlReq, 
                                            tipoAcesso=typeReq, 
                                            head=authorization, 
                                            payload=filters)
        
            # Resultado JSON da requisicao
            result = getReq.json()

            # Checa se esta vazio o resultado, o que significa que nao tem mais paginas
            if (len(result) == 0) or (getReq.status_code != 200):
                    print(">> " + str(getReq.status_code))
                    break
            else:
                    # coloca cada retorno na lista
                    retorno.append(result)
                    i+=1
    # Cada usuario do token fica em uma sublista da lista de retorno, por isto a necessidade de 'explodir' ela dentro de outra lista.
    retornoLista = [item for sublista in retorno for item in sublista]
    return retornoLista 

def requests_API_Orquestra(metodo='GET',urlAcesso="https://elysia.zeev.it",tipoAcesso="/api/2/assignments",head={},payload={}):
    '''
        Funcao
        ------
            Funcao que retorna a requisicao desejada a partir do url de acesso e link de tipo de acesso.
            !Sendo permitido apenas metodos POST e GET!

        Parameters
        ----------

            metodo: str (OPCIONAL)
                Metodo a ser utilizado na requisicao, POST ou GET.
                Se nao for especificado nenhum ele ira utilizar o metodo 'GET'.
            
            urlAcesso: str (OPCIONAL)
                Url usada na requisicao.
                Se nao for especificado nenhum ele ira utilizar o URL que ja esta no funcao.
            
            tipoAcesso: str (OPCIONAL)
                Url de acesso para o tipo de funcao na API, POST ou GET.
                Se nao for especificado nenhum ele ira utilizar o URL que ja esta no funcao.
                Ex.: **/api/2/assignments**
            
            head: str (OBRIGATORIO)
                Autorizacao para acesso da API.
                Ex.: **{'Authorization': 'token_acesso'}**
            
            payload: str (OPCIONAL)
                Parametros para acesso atraves de filtros na requisicao.
        
        Retorno
        -------
            Retorna o objeto response ou resposta de erro caso ocorra.
            Ex.: <Response [200]> 
    '''
    # verifica qual tipo de requisicao foi pedida
    if metodo == 'GET':
        req = requests.get(url=urlAcesso+tipoAcesso,headers=head,params=payload)
        return req
    elif metodo == 'POST':
        req = requests.post(url=urlAcesso+tipoAcesso,headers=head,params=payload)
        return req
    else: 
        raise NameError('Nenhum metodo valido foi passado. Passe os metodos POST ou GET.')
def instances_Report_Orquestra(userToken:str, showFinishedInstanceTasks:bool=True, showPendingInstanceTasks:bool=False, activeInstances:bool=True, forceReq:bool=False, flowId:int=0, periodo:bool=False):
    '''
        Funcao
        ------
            Lista todas instâncias de solicitações que a pessoa relacionada ao token possui permissão de consultar de acordo com filtros.
        
        Parametros
        ----------
            userToken : str, (obrigatorio)
                Token do usuario orquestra para solicitar a requisicao.
            
            Parametros de especial atencao:
            -------------------------------

            `showFinishedTasks` : boolean, (opcional)
                Define se vai requisitar instancias fechadas das solicitacoes.
            
            `showPendingInstanceTasks` : boolean, (opcional)
                Define se vai requisitar instancias abertas das solicitacoes.

            * Se nao optar por `showPendingInstanceTasks` ou `showFinishedTasks`, sera necessario colocar o parametro `forceReq` como `True`.\n
            * Se `showPendingInstanceTasks` ou `showFinishedTasks` estiverem `True` o parametro `forceReq` eh alterado para `False`, mesmo se ele for passado pela funcao.
            
            forceReq : boolean, (opcional)
                Este parametro limita a quantidade de solicitacoes por segundo.

            activeInstances : boolean, (opcional)
                Definir se a solicitacao esta `Finalizada` ou `Em andamento`.

            flowId : str, (opcional)
                Filtrar pelo id de instancia.
            
            periodo : boolean, (opcional)
                Define se o periodo vai ser utilizado
        Retorno 
        -------
            Retorna um DataFrame(Pandas).
            * Se erro, retorna -1.
    '''
    # Filtro padrao - Solicitacoes Em andamento e com instancias Finalizadas
    f = {"startDateIntervalBegin": "2000-01-01T00:00:00",
            "startDateIntervalEnd": "2030-12-31T23:59:59",
            "showFinishedInstanceTasks": showFinishedInstanceTasks, 
            "showPendingInstanceTasks": showPendingInstanceTasks,
            "formFieldNames":["numeroDoProjeto",
                              "cidade",
                              "origemDoCliente",
                              "formatoEvento",
                              "executivoDeVendas",
                              "preVendedor",
                              "nomeDoCliente",
                              "dataDoAgendamento",
                              "observacoes"
                              ],
            "active": activeInstances}

    # Filtro de ID para solicitacoes especificas
    if flowId != 0: 
        f['flowId'] = flowId
    
    # Usar periodo de pesquisa
    if periodo:
        f['startDateIntervalBegin'] = '2023-01-01' + "T00:00:00"
        f['startDateIntervalEnd'] = today + "T23:59:59"
    
    if showFinishedInstanceTasks == True | showPendingInstanceTasks == True:
        forceReq = False
        
    lista = request_Orquestra(method='GET',
                                urlReq=url,
                                typeReq='/api/2/instances/report',
                                tokens=[userToken],
                                filters=f,
                                forceRequisition=forceReq)
    df = pd.DataFrame(lista)
    return df

In [11]:
# ---- Inicio do codigo ----
# Buscar o dados do fluxo 
# Numero do fluxo: 368
instanciasInativas = instances_Report_Orquestra(tokenApi, True, True, False, flowId=368, periodo=True) # Inativas
instanciasAtivas = instances_Report_Orquestra(tokenApi, True, True, True, flowId=368, periodo=True) # Ativas

Requisicao para https://elysia.zeev.it/api/2/instances/report <<
---- >> Requisicao lenta... << ---- 
{'startDateIntervalBegin': '2023-01-01T00:00:00', 'startDateIntervalEnd': '2023-04-04T23:59:59', 'showFinishedInstanceTasks': True, 'showPendingInstanceTasks': True, 'formFieldNames': ['numeroDoProjeto', 'cidade', 'origemDoCliente', 'formatoEvento', 'executivoDeVendas', 'preVendedor', 'nomeDoCliente', 'dataDoAgendamento', 'observacoes'], 'active': False, 'flowId': 368, 'recordsPerPage': 30, 'mobileEnabledOnly': False, 'pageNumber': 1}
{'startDateIntervalBegin': '2023-01-01T00:00:00', 'startDateIntervalEnd': '2023-04-04T23:59:59', 'showFinishedInstanceTasks': True, 'showPendingInstanceTasks': True, 'formFieldNames': ['numeroDoProjeto', 'cidade', 'origemDoCliente', 'formatoEvento', 'executivoDeVendas', 'preVendedor', 'nomeDoCliente', 'dataDoAgendamento', 'observacoes'], 'active': False, 'flowId': 368, 'recordsPerPage': 30, 'mobileEnabledOnly': False, 'pageNumber': 2}
{'startDateIntervalB

In [ ]:
frames = [instanciasAtivas, instanciasInativas]
instancias = pd.concat(frames)
instancias

In [17]:
arr = instancias['formFields'].array
listDict = [dic for dic in arr]
df = pd.DataFrame(columns=['Cliente','Projeto', 'Origem', 'Cidade', 'Executivo','Pré-Venda','Formato','Data da Visita', 'Observações'])
valores = []
index = 0
for i in listDict:
    for dicio in i:
       valores.append(dicio['value'])
    df.loc[index] = valores
    index += 1
    valores = []
df = df[['Projeto', 'Cidade', 'Origem', 'Formato', 'Executivo','Pré-Venda','Cliente','Data da Visita', 'Observações']]
df

,Projeto,Cidade,Origem,Formato,Executivo,Pré-Venda,Cliente,Data da Visita,Observações
0,12021,Novo Hamburgo,Inbound,Presencial,Diego Paz,Heloisa Zanetti,Panelinha da Ju (Juliana Hinterlang),03/04/2023,
1,11541,Gravataí,Inbound,Presencial,Tomás Zanetti,Heloisa Zanetti,GoodYear,03/04/2023,
2,12035,Bom Jesus,Inbound,Online,Diego Paz,Nicolas França,João Carlos Hugen Machado,30/03/2023,
3,11541,Gravataí,Inbound,Presencial,Tomás Zanetti,Heloisa Zanetti,GoodYear,03/04/2023,
4,12041,Porto Alegre,Inbound,Presencial,Tomás Zanetti,Heloisa Zanetti,RecordTV,29/03/2023,
5,12038,Estrela,Inbound,Presencial,Tomás Zanetti,Heloisa Zanetti,Carlos Wermann,24/03/2023,
6,12036,Cachoeirinha,Inbound,Presencial,Breno Bublitz,Heloisa Zanetti,Timóteo Fernandes,27/03/2023,
7,12026,Vacaria,Inbound,Online,Tomás Zanetti,Vinícius Pizzinato,Rodrigo Minella Dipp,17/03/2023,
8,12023,Alvorada,Inbound,Presencial,Tomás Zanetti,Heloisa Zanetti,Pedro Soster (Saf Holland),15/03/2023,
9,12051,Canoas,Próprio,Presencial,Tomás Zanetti,João Aguiar,Marco Bortoluzzii,03/04/2023,
